In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
station_list = pd.read_excel('stationlist.xlsx', header=3,
                            usecols = 'D:G', index_col=-1)
station_list.drop_duplicates(inplace=True)
station_list.head()

In [ ]:
Nbus = ['N13', 'N15','N16','N26','N30','N32','N34','N37'
        ,'N51','N61','N62','N64','N72','N75']
len(Nbus)

In [ ]:
import glob
bus_station_data = glob.glob('정류소*')

In [ ]:
for nb, data in zip(Nbus, bus_station_data):
    globals()[nb] = pd.read_csv(data, sep='\t', index_col = 0)
    globals()[nb].drop(index = '순서', inplace = True)
    globals()[nb]['경유정류소'] = globals()[nb]['경유정류소'].str.replace(' - 회차', '')
    print(nb, globals()[nb].shape) 

In [ ]:
for nb in Nbus:
    snum = pd.to_numeric(globals()[nb]['경유정류소'].str[-6:-1])
    globals()[nb]['snum'] = snum
    globals()[nb]['정류소명'] = globals()[nb]['경유정류소'].str[:-7]

In [ ]:
import folium
from folium import plugins
import json

In [ ]:
color = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'black', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'gray', 'pink']


In [ ]:
st_new = pd.read_excel('20220429기준_서울시정류소리스트.xlsx')

In [ ]:
import googlemaps
gmaps_key = '---'
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
print(station_list.loc[11193, '정류소명'])
print([st_new[st_new['ARS-ID'] == 11193].loc[:, '정류소명']][0].values[0])

In [ ]:
map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)

fg1 = folium.FeatureGroup(name='정류소')
map.add_child(fg1)

fg2 = folium.FeatureGroup(name='구역')
map.add_child(fg2)

fg = folium.FeatureGroup(name='정류소_지점')
map.add_child(fg)

g1 = folium.FeatureGroup('100m')
map.add_child(g1)

for i in range(2, 11):
    globals()['g' + str(i)] = folium.FeatureGroup(str(i*100) + 'm')
    map.add_child(globals()['g' + str(i)])

folium.LayerControl(collapsed=False).add_to(map)

geo_path = '02. skorea_municipalities_geo_simple.json' # 각 '구'의 경계면의 위도-경도를 기록한 파일
geo_str = json.load(open(geo_path, encoding='utf-8'))
err_snum = [] # 누락된 정류소 정보를 담는 리스트
lat_lng = [] # 입력된 정류소들의 위도-경도 정보를 담는 리스트

def make_circle(yx):
    for i in range(1, 11):
            folium.Circle(
                            location=yx,
                            radius=i*100, # 원 크기
                            color = '#ff4f4f', # 원 테두리 색상
                            fill_color='red', # 원 내부 색상
                            fillOpacity = 1.0, # 원 내부의 투명도
            ).add_to(globals()['g' + str(i)])

for nb, cl in zip(Nbus, color):
    for sn in globals()[nb]['snum']:
        bus_number = int(str(nb[1:]))
        icon_num = plugins.BeautifyIcon(border_color=cl, text_color=cl, number=bus_number, inner_icon_style='margin-top:0;')
        folium.CircleMarker(yx,radius = 0.5,color = 'red').add_to(fg)            
        try:
            yx = [station_list.loc[sn, 'Y좌표'], station_list.loc[sn, 'X좌표']]
            folium.Marker(yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,station_list.loc[sn, '정류소명'], sn), 
                                                                                  max_width = 500)).add_to(fg1)
            make_circle(yx)
            lat_lng.append(yx)
        except:
            try:
                yx = [st_new[st_new['ARS-ID'] == sn].loc[:, '좌표Y'].values[0], 
                      st_new[st_new['ARS-ID'] == sn].loc[:, '좌표X'].values[0]]
                folium.Marker(location=yx, icon = icon_num, 
                              popup = folium.Popup('%s: %s: %d'%(nb ,[st_new[st_new['ARS-ID'] == sn].loc[:, '정류소명']][0].values[0], sn),
                                                                                 max_width = 500)).add_to(fg1)
                make_circle(yx)
                lat_lng.append(yx)
            except:
                try:
                    sname = globals()[nb][globals()[nb]['snum'] == sn].iloc[0][2]
                    g = gmaps.geocode(sname + ' 정류장' , language = 'ko') # google API를 이용하여 위도-경도 정보 가져옴
                    geo = g[0].get('geometry').get('location')
                    yx = [geo.get('lat'), geo.get('lng')]
                    folium.Marker(location=yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,sname, sn), 
                                                                                    max_width = 500)).add_to(fg1)
                    make_circle(yx)
                    lat_lng.append(yx)
                except:
                    err_snum.append([nb,sn])
        
            
folium.GeoJson(geo_str).add_to(fg2)
print('누락된 정류소:', err_snum)

map

### 위에서 기록된 정류소 위도-경도를 기반으로, 사각지대에 존재하는 정류소들 찍기

In [ ]:
lat_lng

In [ ]:
len(lat_lng) # 총 1854개의 정류소 정보를 볼 수 있다.

In [ ]:
from haversine import haversine
haversine(lat_lng[0], lat_lng[1]) * 1000

In [ ]:
# haversine 테스트
for i in lat_lng:
    print(haversine(lat_lng[0], i) * 1000 < 1000)

In [ ]:
st_new.head()

In [ ]:
st_new.info() # 10935 entries

In [ ]:
station_list.info() # 12854 entries

In [ ]:
x0 = st_new.loc[0, '좌표X']
y0 = st_new.loc[0, '좌표Y']
print([x0, y0])

In [ ]:
blind_spot_li = []
for i in range(0, 10935):    
    yx = [st_new.loc[i, '좌표Y'], st_new.loc[i, '좌표X']]
    is_blind_spot = 1
    for j in lat_lng:
        if (haversine(yx, j) * 1000) < 500:
            is_blind_spot = 0            
    if is_blind_spot == 1: 
        blind_spot_li.append(yx)
    
print(len(blind_spot_li))    

In [ ]:
blind_spot_li = []
for i in range(0, 10935):    
    yx = [st_new.loc[i, '좌표Y'], st_new.loc[i, '좌표X']]
    is_blind_spot = 1
    for j in lat_lng:
        if (haversine(yx, j) * 1000) < 500:
            is_blind_spot = 0
            break                
    if is_blind_spot == 1: blind_spot_li.append(yx)
    
print(len(blind_spot_li))  

In [ ]:
map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)

fg1 = folium.FeatureGroup(name='정류소')
map.add_child(fg1)

fg2 = folium.FeatureGroup(name='구역')
map.add_child(fg2)

fg = folium.FeatureGroup(name='정류소_지점')
map.add_child(fg)

bs5 = folium.FeatureGroup(name='사각지대_500')
map.add_child(bs5)

g1 = folium.FeatureGroup('100m')
map.add_child(g1)

for i in range(2, 11):
    globals()['g' + str(i)] = folium.FeatureGroup(str(i*100) + 'm')
    map.add_child(globals()['g' + str(i)])

folium.LayerControl(collapsed=False).add_to(map)

geo_path = '02. skorea_municipalities_geo_simple.json' # 각 '구'의 경계면의 위도-경도를 기록한 파일
geo_str = json.load(open(geo_path, encoding='utf-8'))
err_snum = [] # 누락된 정류소 정보를 담는 리스트
lat_lng = [] # 입력된 정류소들의 위도-경도 정보를 담는 리스트

def make_circle(yx):
    for i in range(1, 11):
            folium.Circle(
                            location=yx,
                            radius=i*100, # 원 크기
                            color = '#ff4f4f', # 원 테두리 색상
                            fill_color='red', # 원 내부 색상
                            fillOpacity = 0.5, # 원 내부의 투명도
            ).add_to(globals()['g' + str(i)])

for nb, cl in zip(Nbus, color):
    for sn in globals()[nb]['snum']:
        bus_number = int(str(nb[1:]))
        icon_num = plugins.BeautifyIcon(border_color=cl, text_color=cl, number=bus_number, inner_icon_style='margin-top:0;')
        folium.CircleMarker(yx,radius = 0.5,color = 'red').add_to(fg)            
        try:
            yx = [station_list.loc[sn, 'Y좌표'], station_list.loc[sn, 'X좌표']]
            folium.Marker(yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,station_list.loc[sn, '정류소명'], sn), 
                                                                                  max_width = 500)).add_to(fg1)
            make_circle(yx)
            lat_lng.append(yx)
        except:
            try:
                yx = [st_new[st_new['ARS-ID'] == sn].loc[:, '좌표Y'].values[0], 
                      st_new[st_new['ARS-ID'] == sn].loc[:, '좌표X'].values[0]]
                folium.Marker(location=yx, icon = icon_num, 
                              popup = folium.Popup('%s: %s: %d'%(nb ,[st_new[st_new['ARS-ID'] == sn].loc[:, '정류소명']][0].values[0], sn),
                                                                                 max_width = 500)).add_to(fg1)
                make_circle(yx)
                lat_lng.append(yx)
            except:
                try:
                    sname = globals()[nb][globals()[nb]['snum'] == sn].iloc[0][2]
                    g = gmaps.geocode(sname + ' 정류장' , language = 'ko') # google API를 이용하여 위도-경도 정보 가져옴
                    geo = g[0].get('geometry').get('location')
                    yx = [geo.get('lat'), geo.get('lng')]
                    folium.Marker(location=yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,sname, sn), 
                                                                                    max_width = 500)).add_to(fg1)
                    make_circle(yx)
                    lat_lng.append(yx)
                except:
                    err_snum.append([nb,sn])
        

for spot in blind_spot_li:
    folium.CircleMarker(spot,radius = 0.5,color = 'purple').add_to(bs5)
        
folium.GeoJson(geo_str).add_to(fg2)
print('누락된 정류소:', err_snum)


map    

In [ ]:
s1 = set(list(station_list.index))
s1

In [ ]:
s2 = set(list(st_new.loc[:, 'ARS-ID']))
s2

In [ ]:
print(len(s1), len(s2))
print(len(s1 - s2), len(s2 - s1))

In [ ]:
# find_bs: 사각지대(blind_spot)를 찾는 함수 제작
def find_bs(leng, search_ll , standard_ll): 
    # search_ll : 사각지대 찾고자 하는 리스트, standard_ll : 기준이 되는 리스트
    blind_spot_li = []
    for i in range(0, len(search_ll)):    
        yx = [search_ll.loc[i, '좌표Y'], search_ll.loc[i, '좌표X']]
        is_blind_spot = True
        for j in standard_ll:
            if (haversine(yx, j) * 1000) < leng:
                is_blind_spot = False
                break                
        if is_blind_spot: blind_spot_li.append(yx)    
    return blind_spot_li

In [ ]:
for i in range(1, 10): # 함수 검증용 코드
    c = find_bs(i * 100, st_new, lat_lng)
    print(len(c)) # 거리(100m)별 야간버스 사각지대 안에 있는 정류소 개수, 거리가 늘 수록 감소해야 한다.

In [ ]:
map = folium.Map(location = [37.5502, 126.982], zoom_start = 10.5)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map)

fg1 = folium.FeatureGroup(name='정류소')
map.add_child(fg1)

fg2 = folium.FeatureGroup(name='구역')
map.add_child(fg2)

fg = folium.FeatureGroup(name='정류소_지점')
map.add_child(fg)

for i in range(1, 11): # g1 ~ g10까지 거리별 레이어 생성
    globals()['g' + str(i)] = folium.FeatureGroup(str(i*100) + 'm')
    map.add_child(globals()['g' + str(i)])

folium.LayerControl(collapsed=False).add_to(map) # LayerControl 활성화

geo_path = '02. skorea_municipalities_geo_simple.json' # 각 '구'의 경계면의 위도-경도를 기록한 파일
geo_str = json.load(open(geo_path, encoding='utf-8'))
err_snum = [] # 누락된 정류소 정보를 담는 리스트
#lat_lng = [] # 입력된 정류소들의 위도-경도 정보를 담는 리스트

def make_circle(yx): # 야간 버스 정류소 기준 거리별 구역 시각화 및 각 레이어에 지정
    for i in range(1, 11):
            folium.Circle(
                            location=yx,
                            radius=i*100, # 원 크기(m 단위)
                            color = '#ff4f4f', # 원 테두리 색상
                            fill_color='red', # 원 내부 색상
                            fillOpacity = 0.5, # 원 내부의 투명도
            ).add_to(globals()['g' + str(i)])

for nb, cl in zip(Nbus, color):
    for sn in globals()[nb]['snum']: # 각 노선 별 정류소번호
        bus_number = int(str(nb[1:])) # 노선 번호
        icon_num = plugins.BeautifyIcon(border_color=cl, text_color=cl, 
                                        number=bus_number, inner_icon_style='margin-top:0;') # 노선 번호 기반으로 마커 생성
        try:# 위도-경도 좌표를 구한 후, 지도에 시각화
            yx = [station_list.loc[sn, 'Y좌표'], station_list.loc[sn, 'X좌표']]
            folium.Marker(yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,station_list.loc[sn, '정류소명'], sn), 
                                                                                  max_width = 500)).add_to(fg1)
            make_circle(yx)
            #lat_lng.append(yx)
        except:
            try:
                yx = [st_new[st_new['ARS-ID'] == sn].loc[:, '좌표Y'].values[0], 
                      st_new[st_new['ARS-ID'] == sn].loc[:, '좌표X'].values[0]]
                folium.Marker(location=yx, icon = icon_num, 
                              popup = folium.Popup('%s: %s: %d'%(nb ,[st_new[st_new['ARS-ID'] == sn].loc[:, '정류소명']][0].values[0], sn),
                                                                                 max_width = 500)).add_to(fg1)
                make_circle(yx)
                #lat_lng.append(yx)
            except:
                try:
                    sname = globals()[nb][globals()[nb]['snum'] == sn].iloc[0][2]
                    g = gmaps.geocode(sname + ' 정류장' , language = 'ko') # google API를 이용하여 위도-경도 정보 가져옴
                    geo = g[0].get('geometry').get('location')
                    yx = [geo.get('lat'), geo.get('lng')]
                    folium.Marker(location=yx, icon = icon_num, popup = folium.Popup('%s: %s: %d'%(nb ,sname, sn), 
                                                                                    max_width = 500)).add_to(fg1)
                    make_circle(yx)
                    #lat_lng.append(yx)
                except:
                    err_snum.append([nb,sn])
        folium.CircleMarker(yx,radius = 0.5,color = 'red').add_to(fg) # 정류소 지점 마커로 찍기
for i in range(1, 11):        
    blind_spot_li = find_bs(i * 100, st_new, lat_lng)
    for spot in blind_spot_li: # 사각지대에 해당되는 정류소, 지도에 시각화
        folium.CircleMarker(spot,radius = 0.5,color = 'black').add_to(globals()['g' + str(i)])
        
folium.GeoJson(geo_str).add_to(fg2) # 서울시의 '구' 지도에 시각화
print('누락된 정류소 개수', len(err_snum))

map    

In [ ]:
#########################################################
map.save('../output/NightBusStation_map_2.html')

In [ ]:
station_list['좌표X'] = station_list['X좌표']
station_list['좌표Y'] = station_list['Y좌표']
station_list.reset_index(inplace=True)
station_list.head()

In [ ]:
st_new.head()